In [ ]:
# Import des librairies
import pandas as pd
from models.managers.db_manager import connect_to_bdd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc


In [ ]:
conn = connect_to_bdd()

# Requête pour election_2022_t1 et sauvegarde dans df_election
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Requête pour economie et sauvegarde dans df_economie
query_economie = """
    SELECT avg_1982, avg_1983, avg_1984, avg_1985, avg_1986, avg_1987, avg_1988, avg_1989,
           avg_1990, avg_1991, avg_1992, avg_1993, avg_1994, avg_1995, avg_1996, avg_1997,
           avg_1998, avg_1999, avg_2000, avg_2001, avg_2002, avg_2003, avg_2004, avg_2005,
           avg_2006, avg_2007, avg_2008, avg_2009, avg_2010, avg_2011, avg_2012, avg_2013,
           avg_2014, avg_2015, avg_2016, avg_2017, avg_2018, avg_2019, avg_2020, avg_2021,
           avg_2022, avg_2023, code_postal
    FROM economie
"""
df_economie = pd.read_sql(query_economie, conn)

# Requête pour securite et sauvegarde dans df_securite
query_securite = """
    SELECT code_postal, tauxpourcent
    FROM securite
"""
df_securite = pd.read_sql(query_securite, conn)

# Requête pour densite et sauvegarde dans df_densite
query_densite = """
    SELECT code_postal, dens_pop
    FROM demographie
"""
df_densite = pd.read_sql(query_densite, conn)

# Fermeture de la connexion
conn.close()

In [ ]:
# Fusion des dataframes sur le code_postal
df_final = df_election.merge(df_economie, on='code_postal', how='inner')
df_final = df_final.merge(df_securite, on='code_postal', how='inner')
df_final = df_final.merge(df_densite, on='code_postal', how='inner')


In [ ]:
# Création de la variable cible
df_final['target'] = (df_final['% Vot/Ins'] > 50).astype(int)

In [ ]:
# Sélection des variables explicatives pouvant influencer le vote

features = df_final[['dens_pop', 'avg_2023', 'tauxpourcent']]  # Exemple simplifié, ajoutez d'autres variables pertinentes

In [ ]:
# Création et évaluation du modèle

# Division des données en ensemble d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(features, df_final['target'], test_size=0.3, random_state=42)

# Création du modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print(f'Accuracy: {accuracy}')
print(f'ROC AUC Score: {roc_auc}')

In [ ]:
# Interprétation des résultats

# Affichage des coefficients
coefs = pd.DataFrame({
    'Variable': features.columns,
    'Coefficient': model.coef_[0]
})
print(coefs)


In [ ]:
# Histogramme des probabilités de participation

# Prédictions de probabilité
y_prob = model.predict_proba(X_test)[:, 1]

# Création de l'histogramme
plt.figure(figsize=(8, 6))
sns.histplot(y_prob, bins=30, kde=False, color='blue')
plt.title('Distribution des probabilités de participation supérieure à 50%')
plt.xlabel('Probabilité de participation > 50%')
plt.ylabel('Nombre d\'observations')
plt.show()


In [ ]:
# Courbe ROC

# Calcul des taux pour la courbe ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Création de la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Affichage des coefficients dans un diagramme à barres
coefs = pd.DataFrame({
    'Variable': X_train.columns,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

plt.figure(figsize=(8, 6))
sns.barplot(x='Coefficient', y='Variable', data=coefs)
plt.title('Impact des Variables sur la prédiction')
plt.xlabel('Coefficient')
plt.ylabel('Variable')
plt.show()
